In [1]:
pip install riotwatcher

Note: you may need to restart the kernel to use updated packages.


In [3]:
from riotwatcher import LolWatcher
import json
import pandas as pd
import datetime as dt
lol_watcher = LolWatcher('RGAPI-XXXX-xxxx-XXXX')
my_region = 'la1'



In [4]:
me = lol_watcher.summoner.by_name(my_region, 'SquareHammer')
print(me)
my_ranked_stats = lol_watcher.league.by_summoner(my_region, me['id'])
my_ranked_stats

{'id': 'VD_g_pCYuvPPFOV_waLjp0JBg0dsfipWKKS_7qSJ-M8C', 'accountId': 'vRgfRW5Ebfb4_VMXND8ECORbqXyqcQgkNSoP1NW92vk', 'puuid': 'bKIY8nxkX7Dt4M8GMtOMRHgfmQ71AvJBqPTzrIvLHcdm2jGacY4OoJHEZ5alUS3TAFmch8UZzLQW9w', 'name': 'SquareHammer', 'profileIconId': 4887, 'revisionDate': 1631485043000, 'summonerLevel': 597}


[{'leagueId': 'c0e7ace6-3085-4e33-91b3-f06866df90da',
  'queueType': 'RANKED_SOLO_5x5',
  'tier': 'SILVER',
  'rank': 'II',
  'summonerId': 'VD_g_pCYuvPPFOV_waLjp0JBg0dsfipWKKS_7qSJ-M8C',
  'summonerName': 'SquareHammer',
  'leaguePoints': 58,
  'wins': 139,
  'losses': 155,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False}]

In [103]:
my_matches = lol_watcher.match.matchlist_by_account(my_region, me['accountId'])

# fetch last match detail
last_match = my_matches['matches'][2]
dftest=pd.DataFrame
dftest=pd.json_normalize(my_matches,record_path=['matches'])
dfRanked=pd.DataFrame


dfRanked=dftest.query("role=='SOLO'")

#dfRanked['col1_converetd'] = pd.to_datetime(dt.datetime.now - dfRanked.timestamp )

dfRanked

dfRanked['timestamp']= pd.to_datetime(dfRanked['timestamp'],unit='ms')

dfRanked
     

<ipython-input-103-a75400b65cc1>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRanked['timestamp']= pd.to_datetime(dfRanked['timestamp'],unit='ms')


,platformId,gameId,champion,queue,season,timestamp,role,lane
6,LA1,1138246409,69,420,13,2021-09-16 03:17:19.062,SOLO,MID
9,LA1,1136863611,106,450,13,2021-09-13 03:50:50.081,SOLO,BOTTOM
12,LA1,1135889766,41,430,13,2021-09-11 03:47:18.462,SOLO,TOP
18,LA1,1134480163,10,420,13,2021-09-08 03:35:36.982,SOLO,TOP
37,LA1,1132956665,131,420,13,2021-09-05 01:46:32.036,SOLO,TOP
41,LA1,1131734199,131,420,13,2021-09-02 06:47:43.646,SOLO,TOP
42,LA1,1131679021,41,420,13,2021-09-02 03:58:23.588,SOLO,TOP
45,LA1,1131207931,41,420,13,2021-09-01 03:18:23.078,SOLO,TOP
48,LA1,1130221810,41,420,13,2021-08-30 02:00:50.229,SOLO,TOP
49,LA1,1129928568,41,420,13,2021-08-29 17:42:23.522,SOLO,TOP


In [104]:
def SummonerName(match_summoners,participant):
      #print("Yo recibo"+str(participant))
      participantIdentities=match_summoners
      Names=[]
      for y in participantIdentities['participantIdentities']:
            summoner_row={}
            #print("Yo comparare con"+ str(y['participantId']))
            if participant ==  y['participantId']:
                       return  y['player']['summonerName']
            else:
                  continue
      return 'Nadie'




In [105]:
# First we get the latest version of the game from data dragon
versions = lol_watcher.data_dragon.versions_for_region('na1')['n']['champion']
#print(versions)

static_champ_list = lol_watcher.data_dragon.champions(versions, False, 'en_US')
# champ static list data to dict for looking up

champ_dict = []
for key in static_champ_list['data']:
    champcolection={}
    row = static_champ_list['data'][key]
    champcolection["champion"] = int(row['key'])
    champcolection["ChampionName"] = row['name']
    champ_dict.append(champcolection)
      
df2 = pd.DataFrame(champ_dict)
df2
df2.describe()
df2.dtypes



champion         int64
ChampionName    object
dtype: object

In [106]:
def ChampionName(champion):
   
      for index, z in df2.iterrows():
            #print("Yo recibo "+str(champion)+" Yo comparo "+z['champion'])
            if int(champion) ==  int(z['champion']):
                 return  z['ChampionName']
            else:continue

      return 'Nadie'

In [120]:
participants = []
for index,x in dfRanked.iterrows():
      match_detail = lol_watcher.match.by_id(my_region, int(x['gameId']))
      for row in match_detail['participants']:

        participants_row = {}
        participants_row['gameId'] = x['gameId']
        participants_row['Fecha'] = x['timestamp']
        participants_row['participantId'] = row['stats']['participantId']
        participants_row['SummonerName'] = SummonerName(match_detail,row['stats']['participantId'])
        participants_row['champion'] = row['championId']
        participants_row['spell1'] = row['spell1Id']
        participants_row['spell2'] = row['spell2Id']
        participants_row['win'] = row['stats']['win']
        participants_row['kills'] = row['stats']['kills']
        participants_row['deaths'] = row['stats']['deaths']
        participants_row['assists'] = row['stats']['assists']
        participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
        participants_row['goldEarned'] = row['stats']['goldEarned']
        participants_row['champLevel'] = row['stats']['champLevel']
        participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
        participants_row['item0'] = row['stats']['item0']
        participants_row['item1'] = row['stats']['item1']
        participants_row['item2'] = row['stats']['item2']
        participants_row['item3'] = row['stats']['item3']
        participants_row['item4'] = row['stats']['item4']
        participants_row['item5'] = row['stats']['item5']
        participants_row['item6'] = row['stats']['item6']
        participants.append(participants_row)
        df = pd.DataFrame(participants)
        
participants.clear();

df=pd.merge(df,df2,on='champion')
df=df.sort_values(by=['gameId', 'participantId'])

df.to_csv("ChampsMerged.csv")

In [82]:
items=lol_watcher.data_dragon.items(versions,locale='es_MX')
items_dict=[]    
 
for key in items['data']:
        itemExtraction={}
        row = items['data'][key]
        itemExtraction['Name'] = row['name']
        itemExtraction['item0'] = int(key)
        items_dict.append(itemExtraction)
        

df6 = pd.DataFrame(items_dict)

df6

,Name,item0
0,Botas,1001
1,Amuleto de las Hadas,1004
2,Perla de Rejuvenecimiento,1006
3,Cinturón de Gigante,1011
4,Capa de Agilidad,1018
...,...,...
194,Eclipse,6692
195,Garra del Merodeador,6693
196,Rencor de Serylda,6694
197,Colmillo de Serpiente,6695


In [83]:
df7=pd.merge(df,df6 ,on=['item0'])
#df7

dfaux=df6.rename(columns = {'item0':'item1'}, inplace = False)
#df6

df7=pd.merge(df7,dfaux,on=['item1'])
df7

,gameId,participantId,SummonerName,champion,spell1,spell2,win,kills,deaths,assists,...,totalMinionsKilled,item0,item1,item2,item3,item4,item5,item6,Name_x,Name_y
0,1138246409,1,RipScott,Cho'Gath,4,12,False,5,3,2,...,163,6662,1033,3075,3111,3105,1029,3340,Guantelete de Fuego Escarchado,Manto Anulamagia
1,1136863611,1,CivilCake925,Cho'Gath,4,13,True,10,9,24,...,77,6662,3193,3083,3111,3075,3110,2052,Guantelete de Fuego Escarchado,Armadura Pétrea
2,1136863611,6,CHICLESIT0,Alistar,4,7,False,7,12,41,...,33,3068,3193,3083,3047,3075,3065,2052,Égida de Fuego Solar,Armadura Pétrea
3,1129928568,2,IllidanElMagno,Maokai,4,14,True,11,4,13,...,33,3859,3193,3047,6662,3082,1028,3364,Escudo de Targón,Armadura Pétrea
4,1136863611,10,Shacomiste Bro,Nunu & Willump,32,6,False,10,15,24,...,48,6662,3047,3742,3065,3083,0,2052,Guantelete de Fuego Escarchado,Punteras de Acero Revestidas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,1112043017,10,SquareHammer,Jax,12,4,False,4,13,5,...,210,3153,3181,3078,3047,3044,1028,3363,Espada del Rey Arruinado,Rompecascos
297,1111219893,8,Law Doombringer,Kai'Sa,4,7,True,15,6,5,...,160,3086,3085,3006,6676,6672,3123,3363,Fervor,Huracán de Runaan
298,1111223031,9,ALEZTER LAN,Seraphine,3,4,False,2,3,16,...,208,1052,3504,3114,3111,6617,3107,3363,Libro Amplificador,Pebetero Ardiente
299,1110063367,6,Agesera,Jayce,4,14,True,15,12,11,...,228,6695,3042,6694,6692,3814,6609,3364,Colmillo de Serpiente,Muramaná
